# Robust Inference and Advanced TechniquesThis notebook covers **robust inference methods** for panel data, including robust standard errors, bootstrap, sensitivity analysis, and outlier detection.## What You'll Learn- ✅ 8 types of robust standard errors- ✅ When to use each standard error type- ✅ 4 bootstrap methods for panels- ✅ Sensitivity analysis (leave-one-out, subset stability)- ✅ Outlier detection and influence diagnostics- ✅ Jackknife resampling- ✅ Practical guidelines and comparisons## Table of Contents1. [Introduction](#introduction)2. [Robust Standard Errors](#robust-se)3. [Bootstrap Inference](#bootstrap)4. [Sensitivity Analysis](#sensitivity)5. [Outlier Detection](#outliers)6. [Jackknife Methods](#jackknife)7. [Practical Guidelines](#guidelines)---

## 1. Introduction to Robust Inference {#introduction}### Why Robust Inference?Standard inference assumes:- Homoskedasticity (constant variance)- No serial correlation- No cross-sectional dependence- No outliers**Reality**: These assumptions are often violated!**Solution**: Robust inference methods provide valid inference even when assumptions fail.### What We'll Cover| Method | Purpose | When to Use ||--------|---------|-------------|| **Robust SE** | Adjust for heteroskedasticity/correlation | Always recommended || **Bootstrap** | Distribution-free inference | Small samples, complex estimators || **Sensitivity** | Check robustness to specification | Before finalizing results || **Outliers** | Detect influential observations | Data quality concerns |Let's dive in!

In [ ]:
# Import librariesimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsfrom scipy import statsimport panelbox as pb# Configurationpd.set_option('display.max_columns', None)pd.set_option('display.precision', 4)np.random.seed(42)plt.style.use('seaborn-v0_8-darkgrid')sns.set_palette("husl")print(f"PanelBox version: {pb.__version__}")print("Robust inference toolkit ready!")

In [ ]:
# Load datadata = pb.load_grunfeld()print("Dataset loaded:")print(f"Shape: {data.shape}")data.head()

---## 2. Robust Standard Errors {#robust-se}PanelBox supports **8 types of robust standard errors**.

### 2.1 Standard (Non-Robust) BaselineFirst, let's see standard (non-robust) errors for comparison:

In [ ]:
# Fixed Effects with standard errorsfe_standard = pb.FixedEffects(    formula="invest ~ value + capital",    data=data,    entity_col="firm",    time_col="year")results_standard = fe_standard.fit()print("FIXED EFFECTS - Standard Errors")print("="*60)print(results_standard.summary())# Extract for comparisonse_standard = results_standard.std_errorsprint(f"\nStandard errors:")print(se_standard)

### 2.2 Heteroskedasticity-Robust (HC0-HC3)**White standard errors** adjust for heteroskedasticity.Four variants with different finite-sample adjustments:

In [ ]:
# Compare HC0-HC3hc_types = ['HC0', 'HC1', 'HC2', 'HC3']hc_results = {}for hc in hc_types:    model = pb.FixedEffects(        formula="invest ~ value + capital",        data=data,        entity_col="firm",        time_col="year"    )    result = model.fit(cov_type=hc)    hc_results[hc] = result.std_errors# Create comparison tablehc_comparison = pd.DataFrame(hc_results)hc_comparison['Standard'] = se_standardprint("\nHETEROSKEDASTICITY-ROBUST SE COMPARISON")print("="*60)print(hc_comparison)print("\n💡 Interpretation:")print("- HC0: Basic White SE (no finite-sample correction)")print("- HC1: Degrees of freedom correction")print("- HC2: Leverage-based correction")print("- HC3: More conservative (recommended for small samples)")print("\nGenerally: HC0 < HC1 < HC2 < HC3 (conservativeness)")

### 2.3 Clustered Standard Errors**Clustered SE** allow for arbitrary correlation within clusters.Most common for panel data: cluster by entity.

In [ ]:
# Clustered by entity (firm)fe_clustered = pb.FixedEffects(    formula="invest ~ value + capital",    data=data,    entity_col="firm",    time_col="year")results_clustered = fe_clustered.fit(cov_type='clustered', cluster_entity=True)print("\nCLUSTERED STANDARD ERRORS (by entity)")print("="*60)print(f"\nStandard errors:")print(results_clustered.std_errors)print("\nComparison:")comparison = pd.DataFrame({    'Standard': se_standard,    'Clustered': results_clustered.std_errors})print(comparison)print("\n💡 Clustered SE are typically larger (more conservative)")

### 2.4 Driscoll-Kraay Standard Errors**Driscoll-Kraay SE** robust to:- Heteroskedasticity- Serial correlation- Cross-sectional dependence (spatial correlation)**Recommended** for macro panels (countries, regions).

In [ ]:
# Driscoll-Kraayfe_dk = pb.FixedEffects(    formula="invest ~ value + capital",    data=data,    entity_col="firm",    time_col="year")results_dk = fe_dk.fit(cov_type='driscoll_kraay')print("\nDRISCOLL-KRAAY STANDARD ERRORS")print("="*60)print(f"\nStandard errors:")print(results_dk.std_errors)print("\n💡 Driscoll-Kraay:")print("- Robust to both serial and spatial correlation")print("- Works for T → ∞ (long panels)")print("- Commonly used in macro/finance panels")

### 2.5 Newey-West (HAC) Standard Errors**Newey-West SE** (Heteroskedasticity and Autocorrelation Consistent):- Adjusts for heteroskedasticity- Adjusts for serial correlation up to lag L

In [ ]:
# Newey-West with 2 lagsfe_nw = pb.FixedEffects(    formula="invest ~ value + capital",    data=data,    entity_col="firm",    time_col="year")results_nw = fe_nw.fit(cov_type='newey_west', lags=2)print("\nNEWEY-WEST (HAC) STANDARD ERRORS")print("="*60)print(f"Lags: 2")print(f"\nStandard errors:")print(results_nw.std_errors)print("\n💡 Newey-West:")print("- Choose lags based on data frequency")print("- Rule of thumb: L = floor(4*(T/100)^(2/9))")print("- For T=20: L ≈ 2-3 lags")

### 2.6 Panel-Corrected Standard Errors (PCSE)**PCSE** (Parks 1967):- Accounts for panel-specific heteroskedasticity- Accounts for contemporaneous correlation- Accounts for serial correlation

In [ ]:
# PCSEfe_pcse = pb.FixedEffects(    formula="invest ~ value + capital",    data=data,    entity_col="firm",    time_col="year")results_pcse = fe_pcse.fit(cov_type='pcse')print("\nPANEL-CORRECTED STANDARD ERRORS (PCSE)")print("="*60)print(f"\nStandard errors:")print(results_pcse.std_errors)print("\n💡 PCSE:")print("- Good for: N < T (few entities, many time periods)")print("- Estimates full variance-covariance matrix")print("- Can be unstable if N is large")

### 2.7 Comprehensive ComparisonLet's compare all SE types:

In [ ]:
# Create comprehensive comparisonse_comparison = pd.DataFrame({    'Standard': se_standard,    'HC1': hc_results['HC1'],    'HC3': hc_results['HC3'],    'Clustered': results_clustered.std_errors,    'Driscoll-Kraay': results_dk.std_errors,    'Newey-West': results_nw.std_errors,    'PCSE': results_pcse.std_errors})print("\nCOMPREHENSIVE SE COMPARISON")print("="*70)print(se_comparison.round(4))# Visualizefig, ax = plt.subplots(figsize=(12, 6))x = np.arange(len(se_comparison.index))width = 0.1colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink']for i, col in enumerate(se_comparison.columns):    offset = (i - len(se_comparison.columns)/2) * width    ax.bar(x + offset, se_comparison[col], width, label=col, alpha=0.8, color=colors[i])ax.set_xlabel('Variables', fontsize=12)ax.set_ylabel('Standard Error', fontsize=12)ax.set_title('Comparison of Standard Error Types', fontsize=14, fontweight='bold')ax.set_xticks(x)ax.set_xticklabels(se_comparison.index, rotation=45, ha='right')ax.legend()ax.grid(True, alpha=0.3, axis='y')plt.tight_layout()plt.show()print("\n📊 Observations:")print("- Robust SE typically larger than standard SE")print("- Clustered and Driscoll-Kraay most conservative")print("- Choice affects t-statistics and p-values!")

### When to Use Which SE?| Situation | Recommended SE | Reason ||-----------|---------------|---------|| **Default** | Clustered (entity) | Safe, allows within-entity correlation || **Heteroskedasticity only** | HC1 or HC3 | Simpler, efficient || **Serial correlation** | Newey-West or Driscoll-Kraay | HAC properties || **Macro panels** | Driscoll-Kraay | Handles spatial correlation || **Small N, large T** | PCSE | Efficient estimation || **Short panels (small T)** | HC3 or Clustered | Finite-sample robust |**Rule of thumb**: When in doubt, use **clustered SE** (conservative and safe).

---## 3. Bootstrap Inference {#bootstrap}**Bootstrap** provides distribution-free inference:- No normality assumption needed- Works for complex estimators- Provides confidence intervals and standard errorsPanelBox supports **4 bootstrap methods** for panels.

### 3.1 Pairs Bootstrap**Method**: Resample (i,t) pairs with replacement**Use**: General purpose, maintains dependence structure

In [ ]:
# Pairs bootstrapbootstrap = pb.PanelBootstrap(    model=fe_standard,    method='pairs',    n_bootstrap=1000,    seed=42)bootstrap_results = bootstrap.run()print("PAIRS BOOTSTRAP")print("="*60)print(f"Bootstrap iterations: 1000")print(f"\nBootstrap standard errors:")print(bootstrap_results.std_errors)print(f"\n95% Confidence intervals (percentile method):")print(bootstrap_results.conf_int())print("\nComparison with clustered SE:")comparison_boot = pd.DataFrame({    'Clustered SE': results_clustered.std_errors,    'Bootstrap SE': bootstrap_results.std_errors})print(comparison_boot)

### 3.2 Wild Bootstrap**Method**: Multiplies residuals by random weights**Use**: Heteroskedasticity-robust, preserves X**Advantage**: Doesn't require resampling X (good for small N)

In [ ]:
# Wild bootstrapbootstrap_wild = pb.PanelBootstrap(    model=fe_standard,    method='wild',    n_bootstrap=1000,    seed=42)wild_results = bootstrap_wild.run()print("\nWILD BOOTSTRAP")print("="*60)print(f"\nBootstrap standard errors:")print(wild_results.std_errors)print("\n💡 Wild Bootstrap:")print("- Better for heteroskedasticity")print("- Preserves X (no resampling of regressors)")print("- Good for small N, large T")

### 3.3 Block Bootstrap**Method**: Resample blocks of time periods**Use**: Preserves within-entity serial correlation**Important**: Block length affects results

In [ ]:
# Block bootstrap with block length 5bootstrap_block = pb.PanelBootstrap(    model=fe_standard,    method='block',    block_length=5,    n_bootstrap=1000,    seed=42)block_results = bootstrap_block.run()print("\nBLOCK BOOTSTRAP")print("="*60)print(f"Block length: 5")print(f"\nBootstrap standard errors:")print(block_results.std_errors)print("\n💡 Block Bootstrap:")print("- Preserves serial correlation within blocks")print("- Block length choice matters:")print("  - Too small: Doesn't capture dependence")print("  - Too large: Few effective resamples")print("  - Rule of thumb: L = T^(1/3)")

### 3.4 Residual Bootstrap**Method**: Resample residuals (assumes homoskedasticity)**Use**: If you're confident about homoskedasticity**Advantage**: More efficient under correct assumptions

In [ ]:
# Residual bootstrapbootstrap_resid = pb.PanelBootstrap(    model=fe_standard,    method='residual',    n_bootstrap=1000,    seed=42)resid_results = bootstrap_resid.run()print("\nRESIDUAL BOOTSTRAP")print("="*60)print(f"\nBootstrap standard errors:")print(resid_results.std_errors)print("\n⚠ Caution:")print("- Assumes homoskedasticity!")print("- Invalid if heteroskedasticity present")print("- Use wild bootstrap instead if uncertain")

### 3.5 Bootstrap Comparison

In [ ]:
# Compare all bootstrap methodsbootstrap_comparison = pd.DataFrame({    'Pairs': bootstrap_results.std_errors,    'Wild': wild_results.std_errors,    'Block': block_results.std_errors,    'Residual': resid_results.std_errors,    'Clustered SE': results_clustered.std_errors})print("\nBOOTSTRAP METHODS COMPARISON")print("="*60)print(bootstrap_comparison.round(4))# Plotfig, ax = plt.subplots(figsize=(10, 6))bootstrap_comparison.T.plot(kind='bar', ax=ax, rot=45, alpha=0.8)ax.set_xlabel('Method', fontsize=12)ax.set_ylabel('Standard Error', fontsize=12)ax.set_title('Bootstrap Methods Comparison', fontsize=14, fontweight='bold')ax.legend(title='Variables')ax.grid(True, alpha=0.3, axis='y')plt.tight_layout()plt.show()print("\n📊 Which bootstrap to use?")print("- Pairs: Default, general purpose")print("- Wild: Heteroskedasticity present")print("- Block: Serial correlation present")print("- Residual: Only if homoskedasticity certain")

---## 4. Sensitivity Analysis {#sensitivity}**Sensitivity analysis** checks if results are robust to specification changes.

### 4.1 Leave-One-Out AnalysisCheck if results driven by specific entities:

In [ ]:
# Leave-one-out sensitivitysensitivity = pb.SensitivityAnalysis(fe_standard)loo_results = sensitivity.leave_one_out()print("LEAVE-ONE-OUT ANALYSIS")print("="*60)print(f"\nCoefficient ranges when dropping each firm:")print(loo_results.summary())# Plot coefficient stabilityfig, axes = plt.subplots(1, 2, figsize=(14, 5))for i, var in enumerate(['value', 'capital']):    coef_range = loo_results.coefficient_range(var)        axes[i].scatter(range(len(coef_range)), coef_range, alpha=0.6)    axes[i].axhline(y=results_standard.params[var], color='red', linestyle='--',                     label='Full sample', linewidth=2)    axes[i].set_xlabel('Dropped Firm', fontsize=12)    axes[i].set_ylabel(f'{var} Coefficient', fontsize=12)    axes[i].set_title(f'Leave-One-Out: {var}', fontsize=13, fontweight='bold')    axes[i].legend()    axes[i].grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n💡 Interpretation:")print("- If coefficient stable → Results robust")print("- If large jumps → Driven by specific entities")print("- Investigate outliers if unstable")

### 4.2 Subset Stability AnalysisCheck stability across different subsets:

In [ ]:
# Subset stability (e.g., by time period)stability = sensitivity.subset_stability(split_var='year', n_splits=2)print("\nSUBSET STABILITY ANALYSIS")print("="*60)print(f"\nCoefficients by time period:")print(stability.summary())print("\n💡 Check if coefficients stable over time")print("- Stable → No structural break")print("- Unstable → Consider time interactions or break points")

### 4.3 Influence DiagnosticsIdentify influential observations:

In [ ]:
# Influence diagnosticsinfluence = pb.InfluenceDiagnostics(fe_standard)influence_results = influence.compute()print("\nINFLUENCE DIAGNOSTICS")print("="*60)print(f"\nTop 10 influential observations:")print(influence_results.top_influential(n=10))# Plot influence measuresfig, axes = plt.subplots(1, 2, figsize=(14, 5))# Cook's distanceaxes[0].scatter(range(len(influence_results.cooks_d)),                 influence_results.cooks_d, alpha=0.6)axes[0].axhline(y=4/len(data), color='red', linestyle='--',                 label='Threshold (4/n)')axes[0].set_xlabel('Observation', fontsize=12)axes[0].set_ylabel("Cook's Distance", fontsize=12)axes[0].set_title("Cook's Distance", fontsize=13, fontweight='bold')axes[0].legend()axes[0].grid(True, alpha=0.3)# DFBETASaxes[1].scatter(range(len(influence_results.dfbetas)),                 influence_results.dfbetas[:, 0], alpha=0.6)axes[1].axhline(y=2/np.sqrt(len(data)), color='red', linestyle='--',                 label='Threshold (2/√n)')axes[1].axhline(y=-2/np.sqrt(len(data)), color='red', linestyle='--')axes[1].set_xlabel('Observation', fontsize=12)axes[1].set_ylabel('DFBETAS (value)', fontsize=12)axes[1].set_title('DFBETAS for value coefficient', fontsize=13, fontweight='bold')axes[1].legend()axes[1].grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n💡 Influence measures:")print("- Cook's D: Overall influence on fitted values")print("- DFBETAS: Influence on specific coefficients")print("- Large values → Potential outliers or leverage points")

---## 5. Outlier Detection {#outliers}**Outlier detection** identifies unusual observations.

In [ ]:
# Outlier detectionoutlier_detector = pb.OutlierDetector(fe_standard)outlier_results = outlier_detector.detect()print("OUTLIER DETECTION")print("="*60)print(f"\nOutliers detected: {outlier_results.n_outliers}")print(f"\nOutlier observations:")print(outlier_results.outliers)# Visualize residualsfig, axes = plt.subplots(1, 2, figsize=(14, 5))# Residuals vs fittedaxes[0].scatter(fe_standard.fitted_values, fe_standard.residuals, alpha=0.6)axes[0].axhline(y=0, color='red', linestyle='--')axes[0].set_xlabel('Fitted Values', fontsize=12)axes[0].set_ylabel('Residuals', fontsize=12)axes[0].set_title('Residuals vs Fitted', fontsize=13, fontweight='bold')axes[0].grid(True, alpha=0.3)# Mark outliers in redoutlier_idx = outlier_results.outlier_indicesaxes[0].scatter(fe_standard.fitted_values[outlier_idx],                 fe_standard.residuals[outlier_idx],                 color='red', s=100, alpha=0.7, label='Outliers')axes[0].legend()# Q-Q plotstats.probplot(fe_standard.residuals, dist="norm", plot=axes[1])axes[1].set_title('Q-Q Plot', fontsize=13, fontweight='bold')axes[1].grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n💡 What to do with outliers?")print("1. Investigate: Data entry errors?")print("2. Keep if genuine: Use robust methods")print("3. Report sensitivity: Results with/without outliers")print("4. Consider robust estimation (M-estimators)")

---## 6. Jackknife Methods {#jackknife}**Jackknife** systematically drops observations to estimate variance.

In [ ]:
# Jackknifejackknife = pb.PanelJackknife(fe_standard)jackknife_results = jackknife.run()print("JACKKNIFE ANALYSIS")print("="*60)print(f"\nJackknife standard errors:")print(jackknife_results.std_errors)print("\nComparison:")comparison_jack = pd.DataFrame({    'Standard': se_standard,    'Clustered': results_clustered.std_errors,    'Bootstrap': bootstrap_results.std_errors,    'Jackknife': jackknife_results.std_errors})print(comparison_jack.round(4))print("\n💡 Jackknife vs Bootstrap:")print("- Jackknife: Deterministic, faster")print("- Bootstrap: More flexible, better properties")print("- Often give similar results")

---## 7. Practical Guidelines {#guidelines}### Decision Tree: Which Method to Use?```Choose robust inference method:│├─ Standard errors│  ││  ├─ Default → Clustered (entity)│  ├─ Heteroskedasticity only → HC1 or HC3│  ├─ Serial correlation → Newey-West or Driscoll-Kraay│  ├─ Spatial correlation → Driscoll-Kraay│  └─ Small N, large T → PCSE│├─ Bootstrap│  ││  ├─ General → Pairs│  ├─ Heteroskedasticity → Wild│  ├─ Serial correlation → Block│  └─ Small sample → Wild or Block│├─ Sensitivity│  ││  ├─ Check robustness → Leave-one-out│  ├─ Time stability → Subset analysis│  └─ Influential obs → Influence diagnostics│└─ Outliers   │   ├─ Detect → OutlierDetector   └─ Handle → Report sensitivity```### Recommended Workflow1. ✅ **Start with clustered SE** (conservative default)2. ✅ **Check for violations** (use validation tests)3. ✅ **Apply appropriate SE** based on test results4. ✅ **Run sensitivity analysis** (leave-one-out)5. ✅ **Check for outliers** (influence diagnostics)6. ✅ **Bootstrap if needed** (small samples, complex models)7. ✅ **Report multiple specifications** (show robustness)### Quick Reference Table| Concern | Method | PanelBox Code ||---------|--------|---------------|| **Heteroskedasticity** | HC1/HC3 | `fit(cov_type='HC1')` || **Serial correlation** | Newey-West | `fit(cov_type='newey_west')` || **Spatial correlation** | Driscoll-Kraay | `fit(cov_type='driscoll_kraay')` || **Small sample** | Bootstrap | `PanelBootstrap(method='wild')` || **Outliers** | Sensitivity | `OutlierDetector()` || **Robustness check** | Leave-one-out | `SensitivityAnalysis()` |### Best Practices1. ✅ **Always use robust SE** - Standard SE rarely appropriate2. ✅ **Cluster by entity** - Safe default for panels3. ✅ **Bootstrap for complex models** - GMM, IV, etc.4. ✅ **Check sensitivity** - Don't rely on single specification5. ✅ **Investigate outliers** - But don't automatically drop6. ✅ **Report multiple SE types** - Show robustness in papers---## SummaryYou learned:✅ **8 Robust SE Types**: HC0-HC3, clustered, Driscoll-Kraay, Newey-West, PCSE✅ **4 Bootstrap Methods**: Pairs, wild, block, residual✅ **Sensitivity Analysis**: Leave-one-out, subset stability, influence✅ **Outlier Detection**: Multiple methods and visualization✅ **Jackknife**: Alternative to bootstrap✅ **Practical Guidelines**: When to use what### Key Takeaways1. **Clustered SE** are the safe default for panel data2. **Bootstrap** when you need distribution-free inference3. **Always check sensitivity** to outliers and specification4. **Report multiple methods** to demonstrate robustness### Next Steps- **[05_report_generation.ipynb](./05_report_generation.ipynb)**: Create publication-ready reports- **[03_validation_complete.ipynb](./03_validation_complete.ipynb)**: Comprehensive testing- **[02_dynamic_gmm_complete.ipynb](./02_dynamic_gmm_complete.ipynb)**: Dynamic panels---*Robust inference with confidence using PanelBox!*